In [1]:
import torch
import torch.nn as nn
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tqdm

### dataset 

In [2]:
data = load_wine(as_frame=True)

pandas_df = data['data']
targets= data['target']

display(pandas_df.head())
display(targets)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


0      0
1      0
2      0
3      0
4      0
      ..
173    2
174    2
175    2
176    2
177    2
Name: target, Length: 178, dtype: int32

In [3]:
scaler = StandardScaler()
df = scaler.fit_transform(pandas_df.to_numpy())

X_train, X_test, y_train, y_test = train_test_split(df, targets, test_size=0.3, shuffle=True)

In [4]:
tensor_X_train = torch.from_numpy(X_train).to(torch.float32)
tensor_y_train = torch.from_numpy(y_train.to_numpy()).to(torch.int8)

display(tensor_X_train)
display(tensor_X_train.shape)
display(tensor_y_train)
display(tensor_y_train.shape)

tensor([[-0.8284, -1.2086, -1.5225,  ...,  0.4499, -0.4261,  0.0099],
        [ 1.6915, -0.3468,  0.4879,  ..., -0.4275,  1.1841,  2.3346],
        [-1.7795, -0.2570,  3.1563,  ..., -0.1204,  1.5231, -0.8977],
        ...,
        [ 0.2957,  1.4754, -0.2797,  ..., -0.6030,  0.5485, -0.2130],
        [ 2.1610, -0.5443,  0.0858,  ...,  1.2835,  0.1671,  1.2837],
        [-0.4949, -0.8944, -1.7053,  ...,  0.3622,  0.2236, -0.5888]])

torch.Size([124, 13])

tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        2, 2, 1, 0, 0, 1, 0, 1, 2, 0, 0, 1, 0, 2, 0, 2, 0, 0, 2, 0, 1, 1, 1, 1,
        1, 0, 1, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 2, 0, 1, 2,
        0, 2, 2, 2, 2, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 1, 0, 0, 2, 0, 0, 0,
        0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 0, 1, 2, 2, 2, 1,
        1, 0, 0, 1], dtype=torch.int8)

torch.Size([124])

In [5]:
train_dataset = torch.utils.data.TensorDataset(tensor_X_train, tensor_y_train)
for i in train_dataset:
    print(i)
    break

(tensor([-0.8284, -1.2086, -1.5225, -1.4098,  2.5458, -0.6331, -0.1800, -0.0955,
         2.0484, -0.7172,  0.4499, -0.4261,  0.0099]), tensor(1, dtype=torch.int8))


In [6]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
for i in train_dataloader:
    print(i)
    break

[tensor([[ 0.5057,  1.3498, -0.9011, -0.2087, -0.6840,  0.2482,  0.6533, -0.7401,
         -0.1943, -0.3366, -0.2082,  0.5485,  0.9175],
        [ 1.0122, -0.5263,  0.1955, -1.6500,  0.7905,  2.5395,  1.7175, -0.3373,
          0.4890,  0.8616,  0.2306,  0.9157,  1.4111],
        [ 0.6910, -0.5443,  0.3417,  0.3018,  1.1416,  1.0654,  0.7537, -1.3042,
          1.5052,  0.5156,  0.0989,  0.5908,  1.1882],
        [ 0.1969,  0.0212,  1.1093, -0.2687,  0.0884,  0.8090,  1.2155, -0.4984,
          2.1360,  0.2690,  0.3183,  0.7886,  1.3951]]), tensor([0, 0, 0, 0], dtype=torch.int8)]


### model 

In [7]:
model = nn.Sequential(nn.Linear(13, 32), 
                      nn.ReLU(), 
                      nn.Linear(32, 32),
                      nn.ReLU(), 
                      nn.Linear(32,3))

model

Sequential(
  (0): Linear(in_features=13, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=3, bias=True)
)

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [9]:
epochs = 50

for epoch in tqdm.tqdm(range(epochs)):
    for x, y in train_dataloader:
        outputs = model(x)
        tgs = y.to(torch.long)
        
        optimizer.zero_grad()
        loss = criterion(outputs, tgs)
        loss.backward()
        optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 31.88it/s]


In [10]:
model(train_dataset[0][0].unsqueeze(dim=0))

tensor([[-1.7448,  5.4283, -3.9582]], grad_fn=<AddmmBackward0>)

### eval 

In [11]:
with torch.no_grad():
    result = model(torch.from_numpy(X_test).to(torch.float32))
    
result[0:5]

tensor([[-1.3753, -4.0901,  9.7981],
        [-2.4475,  6.5680, -4.5005],
        [-2.6653,  4.7452, -1.8684],
        [-1.0555, -4.3832, 10.1497],
        [-2.2535, -0.6885,  5.5916]])

In [12]:
torch.argmax(result, dim=1)

tensor([2, 1, 1, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2,
        0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 1, 1, 1, 1, 2, 2, 0, 1,
        1, 0, 0, 2, 2, 1])

In [13]:
torch.from_numpy(y_test.to_numpy()).to(torch.int64)

tensor([2, 1, 1, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 1, 0, 2,
        0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 1, 1, 1, 1, 2, 2, 0, 1,
        1, 0, 0, 2, 2, 1])

In [14]:
torch.argmax(result, dim=1) == torch.from_numpy(y_test.to_numpy()).to(torch.int64)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True])